In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [28]:
data = pd.read_csv("ppi.csv",index_col=0)

In [29]:
data.describe()

,domain,aa_ProtPosition,Rlength,normalized_length,normalized_abs_surf_acc,normalized_hydropathy_index,rel_surf_acc,prob_sheet,prob_helix,prob_coil,...,9_wm_pssm_K,9_wm_pssm_M,9_wm_pssm_F,9_wm_pssm_P,9_wm_pssm_S,9_wm_pssm_T,9_wm_pssm_W,9_wm_pssm_Y,9_wm_pssm_V,p_interface
count,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,...,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000,65150.000000
mean,0.596700,185.734091,370.468181,0.509625,0.158091,0.462944,0.206745,0.094659,0.187057,0.718145,...,0.353332,0.321936,0.252415,0.279223,0.389421,0.362017,0.180000,0.256189,0.324617,0.120414
std,0.490564,141.000605,159.158935,0.236844,0.089158,0.336863,0.110228,0.089142,0.178178,0.169518,...,0.169218,0.133116,0.133980,0.202496,0.159900,0.139868,0.145721,0.139781,0.124491,0.325448
min,0.000000,1.000000,38.000000,0.014881,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000144,0.001099,0.000361,0.000051,0.003611,0.003374,0.000106,0.000258,0.004092,0.000000
25%,0.000000,72.000000,231.000000,0.302083,0.101837,0.111111,0.132000,0.043000,0.056000,0.648000,...,0.231721,0.228733,0.149303,0.119889,0.275571,0.264229,0.072451,0.147312,0.239245,0.000000
50%,1.000000,153.000000,362.000000,0.497024,0.144073,0.411111,0.182000,0.086000,0.118000,0.732000,...,0.352786,0.315672,0.238542,0.234119,0.381730,0.362598,0.142969,0.238159,0.321720,0.000000
75%,1.000000,272.000000,496.000000,0.696429,0.195684,0.777778,0.255000,0.142000,0.268000,0.844000,...,0.465291,0.406342,0.335204,0.397679,0.500000,0.457689,0.247493,0.344324,0.403071,0.000000
max,1.000000,695.000000,695.000000,0.992560,1.002910,1.000000,0.918000,0.983000,0.988000,0.999000,...,0.958936,0.866744,0.936737,0.962526,0.928647,0.878450,0.964222,0.901056,0.912698,1.000000


In [3]:
for i,col in enumerate(data.columns):
    print(i,col)

0 domain
1 aa_ProtPosition
2 uniprot_id
3 Rlength
4 sequence
5 normalized_length
6 normalized_abs_surf_acc
7 normalized_hydropathy_index
8 rel_surf_acc
9 prob_sheet
10 prob_helix
11 prob_coil
12 pssm_A
13 pssm_R
14 pssm_N
15 pssm_D
16 pssm_C
17 pssm_Q
18 pssm_E
19 pssm_G
20 pssm_H
21 pssm_I
22 pssm_L
23 pssm_K
24 pssm_M
25 pssm_F
26 pssm_P
27 pssm_S
28 pssm_T
29 pssm_W
30 pssm_Y
31 pssm_V
32 3_wm_normalized_abs_surf_acc
33 5_wm_normalized_abs_surf_acc
34 7_wm_normalized_abs_surf_acc
35 9_wm_normalized_abs_surf_acc
36 3_wm_normalized_hydropathy_index
37 5_wm_normalized_hydropathy_index
38 7_wm_normalized_hydropathy_index
39 9_wm_normalized_hydropathy_index
40 3_wm_rel_surf_acc
41 5_wm_rel_surf_acc
42 7_wm_rel_surf_acc
43 9_wm_rel_surf_acc
44 3_wm_prob_sheet
45 5_wm_prob_sheet
46 7_wm_prob_sheet
47 9_wm_prob_sheet
48 3_wm_prob_helix
49 5_wm_prob_helix
50 7_wm_prob_helix
51 9_wm_prob_helix
52 3_wm_prob_coil
53 5_wm_prob_coil
54 7_wm_prob_coil
55 9_wm_prob_coil
56 3_wm_pssm_A
57 3_wm_pssm_

In [30]:
data = data[data.sequence != 'X']
data.describe()

,domain,aa_ProtPosition,Rlength,normalized_length,normalized_abs_surf_acc,normalized_hydropathy_index,rel_surf_acc,prob_sheet,prob_helix,prob_coil,...,9_wm_pssm_K,9_wm_pssm_M,9_wm_pssm_F,9_wm_pssm_P,9_wm_pssm_S,9_wm_pssm_T,9_wm_pssm_W,9_wm_pssm_Y,9_wm_pssm_V,p_interface
count,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,...,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000
mean,0.596709,185.736773,370.466715,0.509623,0.158093,0.462943,0.206748,0.094660,0.187060,0.718156,...,0.353330,0.321937,0.252416,0.279219,0.389419,0.362014,0.180000,0.256190,0.324616,0.120416
std,0.490562,141.000025,159.159716,0.236845,0.089156,0.336866,0.110226,0.089142,0.178178,0.169496,...,0.169219,0.133117,0.133981,0.202495,0.159899,0.139867,0.145722,0.139782,0.124492,0.325450
min,0.000000,1.000000,38.000000,0.014881,0.004628,0.000000,0.009000,0.000000,0.000000,0.012000,...,0.000144,0.001099,0.000361,0.000051,0.003611,0.003374,0.000106,0.000258,0.004092,0.000000
25%,0.000000,72.000000,231.000000,0.302083,0.101840,0.111111,0.132000,0.043000,0.056000,0.648000,...,0.231721,0.228733,0.149301,0.119889,0.275571,0.264229,0.072451,0.147312,0.239244,0.000000
50%,1.000000,153.000000,362.000000,0.497024,0.144073,0.411111,0.182000,0.086000,0.118000,0.732000,...,0.352786,0.315672,0.238556,0.234119,0.381730,0.362598,0.142969,0.238159,0.321720,0.000000
75%,1.000000,272.000000,496.000000,0.696429,0.195684,0.777778,0.255000,0.142000,0.268000,0.844000,...,0.465291,0.406342,0.335211,0.397679,0.500000,0.457689,0.247493,0.344324,0.403071,0.000000
max,1.000000,695.000000,695.000000,0.992560,1.002910,1.000000,0.918000,0.983000,0.988000,0.999000,...,0.958936,0.866744,0.936737,0.962526,0.928647,0.878450,0.964222,0.901056,0.912698,1.000000


In [5]:
data.sequence=data.sequence.astype('category')
print(data.sequence.dtype)

category


In [6]:
len(list(data.columns))

137

In [7]:
edata=data
cate_features=[4]
enc = OneHotEncoder(handle_unknown='ignore')
column_transformer = ColumnTransformer(
[('encoder', enc, cate_features)],
remainder='passthrough',
verbose_feature_names_out=False
)
edata = column_transformer.fit_transform(edata)
encoded_df = pd.DataFrame(edata, columns=column_transformer.get_feature_names_out(list(data.columns)))
encoded_df.describe()


,sequence_A,sequence_C,sequence_D,sequence_E,sequence_F,sequence_G,sequence_H,sequence_I,sequence_K,sequence_L,...,9_wm_pssm_K,9_wm_pssm_M,9_wm_pssm_F,9_wm_pssm_P,9_wm_pssm_S,9_wm_pssm_T,9_wm_pssm_W,9_wm_pssm_Y,9_wm_pssm_V,p_interface
count,65149.0,65149.0,65149.0,65149.0,65149.0,65149.0,65149.0,65149.0,65149.0,65149.0,...,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149
unique,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,18429.000000,16415.000000,22990.000000,23998.000000,12407.000000,12031.000000,24984.000000,20095.000000,16749.000000,2
top,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
freq,59741.0,64123.0,61529.0,60608.0,62743.0,60695.0,63543.0,61866.0,61555.0,58924.0,...,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,57304


In [8]:
encoded_df

,sequence_A,sequence_C,sequence_D,sequence_E,sequence_F,sequence_G,sequence_H,sequence_I,sequence_K,sequence_L,...,9_wm_pssm_K,9_wm_pssm_M,9_wm_pssm_F,9_wm_pssm_P,9_wm_pssm_S,9_wm_pssm_T,9_wm_pssm_W,9_wm_pssm_Y,9_wm_pssm_V,p_interface
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.302858,0.436017,0.238759,0.10398,0.105653,0.161544,0.060391,0.140326,0.481904,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.286923,0.374128,0.415754,0.109332,0.351726,0.330383,0.241807,0.334863,0.38173,1
65145,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,1
65146,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,1
65147,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,1


In [9]:
y=encoded_df.uniprot_id
y=y.drop_duplicates()
y

0        A0A024RAV5
188          Q503K9
287          Q9I4X1
635          P9WQ67
1033         U3J0P1
            ...    
64887        Q5SIT3
64956        Q38503
65012        Q13794
65066        P02881
65111        P0A427
Name: uniprot_id, Length: 228, dtype: object

In [10]:
y1=y.sample(frac=1)

In [11]:
y1.iloc[:45]

13567    O75807
5343     Q91UM2
47452    P67087
12784    P0A734
64665    Q7Y5B1
9964     Q2G542
62182    Q663P0
50979    Q0I165
62051    P51460
53464    Q8A336
53891    D9PY12
34056    O14733
64156    L7N6F8
46874    P06229
51215    Q12163
16781    Q9Y5P4
40177    Q94IF5
61364    Q9JYH5
63481    Q47785
10993    Q9Z2X2
31473    B8HDZ1
31918    Q7Z6B0
21486    Q9BZE9
57335    P40029
48306    P11086
63378    Q00459
58939    P01180
52594    Q9BT30
24694    O69787
46288    Q9C8L4
43219    P77173
27704    P16778
35674    Q3U4G3
60642    Q9HU04
49144    Q9P7M3
29626    Q01786
41212    O95400
65111    P0A427
43874    Q9HV00
57702    O25094
62441    P13081
37992    Q84527
64423    Q1MVD4
64816    P54494
4399     C6KSZ2
Name: uniprot_id, dtype: object

In [12]:
test_prots1=list(y1.iloc[:45])
train_prots1=list(y1.iloc[45:])

In [13]:
len(test_prots1)


45

In [54]:
len(train_prots1)

183

In [14]:
test1 = encoded_df[encoded_df['uniprot_id'].isin(test_prots1)]
train1 = encoded_df[encoded_df['uniprot_id'].isin(train_prots1)]

In [15]:
test1.p_interface.value_counts()

p_interface
0    10759
1     1523
Name: count, dtype: int64

In [16]:
encoded_df.p_interface.value_counts()

p_interface
0    57304
1     7845
Name: count, dtype: int64

In [17]:
train1.to_csv('train1.csv', index=False)
test1.to_csv('test1.csv',index=False)

In [64]:
x_train1 = train1.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_train1 = train1.p_interface
x_test1 = test1.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_test1 = test1.p_interface

In [21]:
rf = RandomForestClassifier()
rf.fit(x_train1, y_train1)
rf_predictions = rf.predict(x_test1)
print("Accuracy of Random Forest:", accuracy_score(y_test1, rf_predictions))

Accuracy of Random Forest: 0.8763718528082634


In [22]:
y_rf_prob = rf.predict_proba(x_test1)[:, 1]
roc_rf = roc_curve(y_test1, y_rf_prob)
print("AUC of Random Forest:", auc(roc_rf[0], roc_rf[1]))

AUC of Random Forest: 0.6392278020235725


In [25]:
xgboost = XGBClassifier()
xgboost.fit(x_train1, y_train1)
xgb_predictions = xgboost.predict(x_test1)
print("Accuracy of XGBoost:", accuracy_score(y_test1, xgb_predictions))

Accuracy of XGBoost: 0.8663653970303422


In [34]:
y_xgb_prob = xgboost.predict_proba(x_test1)[:, 1]
roc_xgb = roc_curve(y_test1, y_xgb_prob)
print("AUC of XGBoost:", auc(roc_xgb[0], roc_xgb[1]))

AUC of XGBoost: 0.7028069942539226


In [33]:
import joblib
joblib.dump(rf,'rf1.pkl')
joblib.dump(xgboost,'xgb1.pkl')

['xgb1.pkl']

In [4]:
cluster_data=data

In [5]:
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['A','G','V']),1)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['I','L','F','P']),2)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['Y','M','T','S']),3)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['H','N','Q','W']),4)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['R','K']),5)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['D','E']),6)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"]=='C',7)

In [9]:
cluster_data.sequence

dtype('O')

In [13]:
cate_features=[4]
enc = OneHotEncoder(handle_unknown='ignore')
column_transformer = ColumnTransformer(
[('encoder', enc, cate_features)],
remainder='passthrough',
verbose_feature_names_out=False
)
cluster_data = column_transformer.fit_transform(cluster_data)
encoded_df = pd.DataFrame(cluster_data, columns=column_transformer.get_feature_names_out(list(data.columns)))
encoded_df.describe()

,sequence_1,sequence_2,sequence_3,sequence_4,sequence_5,sequence_6,sequence_7,domain,aa_ProtPosition,uniprot_id,...,9_wm_pssm_K,9_wm_pssm_M,9_wm_pssm_F,9_wm_pssm_P,9_wm_pssm_S,9_wm_pssm_T,9_wm_pssm_W,9_wm_pssm_Y,9_wm_pssm_V,p_interface
count,65149.0,65149.0,65149.0,65149.0,65149.0,65149.0,65149.0,65149,65149,65149,...,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149.000000,65149
unique,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,695,228,...,18429.000000,16415.000000,22990.000000,23998.000000,12407.000000,12031.000000,24984.000000,20095.000000,16749.000000,2
top,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,P23945,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
freq,51146.0,49693.0,53632.0,57433.0,57879.0,56988.0,64123.0,38875,228,695,...,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,57304


In [10]:
train1 = pd.read_csv("train1.csv")
test1 = pd.read_csv("test1.csv")

In [37]:
test_prots2=list(test1.uniprot_id.drop_duplicates())
train_prots2=list(train1.uniprot_id.drop_duplicates())

In [14]:
test2 = encoded_df[encoded_df['uniprot_id'].isin(test_prots2)]
train2 = encoded_df[encoded_df['uniprot_id'].isin(train_prots2)]

In [15]:
test2.sequence_1.dtype

dtype('O')

In [26]:
test2.p_interface.value_counts()

p_interface
0    10759
1     1523
Name: count, dtype: int64

In [28]:
train2.to_csv('train2.csv', index=False)
test2.to_csv('test2.csv',index=False)

In [16]:
ordinal_data=data

In [19]:
ordinal_data["sequence"] = ordinal_data["sequence"].mask(ordinal_data["sequence"].isin(['A','G','V']),1.0)
ordinal_data["sequence"] = ordinal_data["sequence"].mask(ordinal_data["sequence"].isin(['I','L','F','P']),2.0)
ordinal_data["sequence"] = ordinal_data["sequence"].mask(ordinal_data["sequence"].isin(['Y','M','T','S']),3.0)
ordinal_data["sequence"] = ordinal_data["sequence"].mask(ordinal_data["sequence"].isin(['H','N','Q','W']),4.0)
ordinal_data["sequence"] = ordinal_data["sequence"].mask(ordinal_data["sequence"].isin(['R','K']),5.0)
ordinal_data["sequence"] = ordinal_data["sequence"].mask(ordinal_data["sequence"].isin(['D','E']),6.0)
ordinal_data["sequence"] = ordinal_data["sequence"].mask(ordinal_data["sequence"]=='C',7.0)

In [22]:
ordinal_data.sequence.astype('float64')

0        3.0
1        3.0
2        6.0
3        3.0
4        5.0
        ... 
65145    2.0
65146    6.0
65147    1.0
65148    6.0
65149    1.0
Name: sequence, Length: 65149, dtype: float64

In [23]:
test3 = ordinal_data[ordinal_data['uniprot_id'].isin(test_prots2)]
train3 = ordinal_data[ordinal_data['uniprot_id'].isin(train_prots2)]

In [24]:
test3.p_interface.value_counts()

p_interface
0    10759
1     1523
Name: count, dtype: int64

In [26]:
train3.to_csv('train3.csv', index=False)
test3.to_csv('test3.csv',index=False)

In [31]:
removed_data=data

In [35]:
removed_data=removed_data.drop('sequence',axis=1)

In [38]:
test4 = removed_data[removed_data['uniprot_id'].isin(test_prots2)]
train4 = removed_data[removed_data['uniprot_id'].isin(train_prots2)]

In [39]:
test4.p_interface.value_counts()

p_interface
0    10759
1     1523
Name: count, dtype: int64

In [41]:
train4.to_csv('train4.csv', index=False)
test4.to_csv('test4.csv',index=False)